In [1]:
from pyrep.pyrep import PyRep
from pyrep.objects.joint import Joint
from pyrep.objects.object import Object
from pyrep.objects.dummy import Dummy
from pyrep.objects.vision_sensor import VisionSensor
from pyrep.objects.proximity_sensor import ProximitySensor
from pyrep.objects.shape import Shape
import transforms3d.quaternions as quaternions
import transforms3d.euler as euler

import pandas as pd 
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
import pickle
import pywt
from numpy import random


In [2]:
def posquat2Matrix(pos, quat):
    T = np.eye(4)
    T[0:3, 0:3] = quaternions.quat2mat([quat[-1], quat[0], quat[1], quat[2]])
    T[0:3, 3] = pos

    return np.array(T)

def matrix2posquat(T):
    pos = T[0:3, 3]
    quat = quaternions.mat2quat(T[0:3, 0:3])
    quat = [quat[1], quat[2], quat[3], quat[0]]

    return np.array(pos), np.array(quat)
    
    
def setPoseAtEE(pos, quat, jawAngle):
    
    theta = 0.4106*jawAngle

    b_T_ee = posquat2Matrix(pos, quat)

    #Taken from simulator. This is a weird transform
    ee_T_sx = np.array([[ 9.99191168e-01,  4.02120491e-02, -5.31786338e-06,4.17232513e-07],
                   [-4.01793160e-02,  9.98383134e-01,  4.02087139e-02, -1.16467476e-04],
                   [ 1.62218404e-03, -4.01759782e-02,  9.99191303e-01, -3.61323357e-04],
                   [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,1.00000000e+00]])

    ee_T_dx = np.array([[-9.99191251e-01, -4.02099858e-02, -1.98098369e-06, 4.17232513e-07],
                   [-4.01773877e-02,  9.98383193e-01, -4.02091818e-02, -1.16467476e-04],
                   [ 1.61878841e-03, -4.01765831e-02, -9.99191284e-01, -3.61323357e-04],
                   [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
    #These transforms do NOT compensate super well yet... 
    #More investigation needs to be done.

    b_T_sx = np.dot(b_T_ee, ee_T_sx)
    b_T_dx = np.dot(b_T_ee, ee_T_dx)

    ct = np.cos(theta)
    st = np.sin(theta)

    #Rotate clockwise about z by theta
    x_T_ts = np.array([[ct, -st, 0,  -st*0.009,],
                     [st,    ct, 0,   ct*0.009],
                     [0 ,   0, 1, 0,],
                     [0,    0, 0, 1]])

    # #Rotate counter clockwise about z by theta
    # dx_T_td = np.array([[ct, st, 0, 0],
    # 				 [-st,ct, 0, 0],
    # 				 [0 ,   0, 1, 0],
    # 				 [0,    0, 0, 1]])
    pos_sx, quat_sx = matrix2posquat(np.dot(b_T_sx, x_T_ts))
    pos_dx, quat_dx = matrix2posquat(np.dot(b_T_dx, x_T_ts))
    # #Translate to the end of the jaw:
    # pos_sx = [pos_sx[0], pos_sx[1], pos_sx[2] - 0.009]
    # pos_dx = [pos_dx[0], pos_dx[1], pos_dx[2]]
    
    return np.hstack((pos_sx, quat_sx)),np.hstack((pos_dx, quat_dx))

In [3]:
pr = PyRep()
pr.launch('dvrk_rendering_feature_opt.ttt')
pr.start()

In [4]:
cam_center = Dummy("cam_center")
cam_center_near = Dummy("cam_near")
init_point = Dummy("init_point")
init_point_pos = init_point.get_position()

dx = Dummy("IK_target_dx_PSM1")
sx = Dummy("IK_target_sx_PSM1")

EE = Dummy("EE_virtual_TOOL1")

l_cam = VisionSensor("Vision_sensor_left")
track_cam = VisionSensor("tracking_cam")

cam_pos = cam_center.get_position()
current_pos = sx.get_position()

In [5]:
# set to cam center
sx.set_position(cam_center_near.get_position())
dx.set_position(cam_center_near.get_position())
pr.step()
sx_init_pose = sx.get_pose()
dx_init_pose = dx.get_pose()

sx_pose_list = []
dx_pose_list = []

In [6]:
# set to initial position
sx.set_pose(sx_init_pose)
dx.set_pose(dx_init_pose)
pr.step()

In [7]:
# open gripper
sx_pose,dx_pose = setPoseAtEE(EE.get_pose()[:3], EE.get_pose()[3:], np.pi/3)
sx.set_pose(sx_pose)
dx.set_pose(dx_pose)
pr.step()
time.sleep(1)
sx_pose,dx_pose = setPoseAtEE(EE.get_pose()[:3], EE.get_pose()[3:], -30)
sx.set_pose(sx_pose)
dx.set_pose(dx_pose)
pr.step()

In [8]:
for i in range(100):
    random_offset = random.normal(loc=0.0, scale=[0.1,0.1,0.06], size=(3))
    sx.set_orientation(random_offset,sx)
    dx.set_orientation(random_offset,dx)
    sx_pos = sx.get_position()
    dx_pos = dx.get_position()
    random_offset = random.normal(loc=0.0, scale=[0.001,0.001,0.0015], size=(3))
    sx_pos += random_offset
    dx_pos += random_offset
    sx.set_position(sx_pos)
    dx.set_position(dx_pos)
    
    pr.step()
    sx_pose_list.append(sx.get_pose())
    dx_pose_list.append(dx.get_pose())
    time.sleep(0.01)

# delete last 100 moves

In [10]:
sx_pose_list = sx_pose_list[:-100]
dx_pose_list = dx_pose_list[:-100]

# play recorded trajectory

In [11]:
for i in range(len(sx_pose_list)):
    sx.set_pose(sx_pose_list[i])
    dx.set_pose(dx_pose_list[i])
    pr.step()
    time.sleep(0.01)

In [208]:
len(sx_pose_list)

3830

# save recorded trajectory

In [211]:
sx_pose_np = np.array(sx_pose_list)
dx_pose_np = np.array(dx_pose_list)

In [212]:
np.save("sx_pose.npy",sx_pose_np)
np.save("dx_pose.npy",dx_pose_np)